# Text 1: Vector space models
**Internet Analytics - Lab 4**

---

**Group:** *W*

**Names:**

* *Cloux Olivier*
* *Reiss Saskia*
* *Urien Thibault*

---

#### Instructions

*This is a template for part 1 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [ ]:
import pickle as pk
import numpy as np
import string
import re
import nltk
import time

from scipy.sparse import csr_matrix
from utils import load_json, load_pkl

from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer

from lab04_helper import *

In [ ]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [ ]:
courses = load_json('data/courses.txt') 
stopwords = load_pkl('data/stopwords.pkl')

In [ ]:
def pickleDump(filename, value):
    with open(filename, "wb") as f:
        pk.dump(value, f)
        
def listPrettyPrint(l):
    
    for a,b,c,d in zip(l[::4],l[1::4],l[2::4],l[3::4]):
        print('{:<30}{:<30}{:<30}{:<}'.format(a,b,c,d))

## Exercise 4.1: Pre-processing

See lab_04 helper.
To avoid error we prefered to be able to have the preprocessing function also available for the part 2 : lsi to perfor term search. 

# TODO
explain why those functions

In [ ]:
#Creation of a dictionary that contains :
#courses ID as keys
#a 3-tuple(uniqueIndex, title, list[separated words]) as value
descDict = dict() 
indexCourse = dict()
index = 0
for i in courses:
    if i['courseId'] not in descDict.keys():
        descDict[i['courseId']] = (index, i['name'], cleaner(i))
        indexCourse[index] = i['courseId']
        index += 1

pickleDump(r"cidWithBag.txt", descDict)
pickleDump(r"indexToCourse.txt", indexCourse)

In [ ]:
ixWords = sorted(descDict['COM-308'][2])
print("Words for Internet Analytics course are (in alphabetical order) :")
listPrettyPrint(ixWords)

In [ ]:
#Creation of 2 dictionary.
#wordIndex contains all distinct words as keys and their unique index as value
#indexWord is the exact opposite. 
wordIndex = dict() 
index = 0
for i in descDict:
    for word in descDict[i][2]:
        if word not in wordIndex.keys():
            wordIndex[word] = index
            index += 1;

indexWord = dict((v, k) for k, v in wordIndex.items())
assert(len(indexWord) == len(wordIndex))
with open(r"indexToWord", "wb") as f:
    pk.dump(indexWord, f)

In [ ]:
occValues = []
occRow = [] #indices of words
occCol = [] #indices of courses
i = 0
for cid in descDict:
    cIndex = descDict[cid][0]
    for word in descDict[cid][2]:
        occCol.append(cIndex)
        occRow.append(wordIndex[word])
        occValues.append(1)
occurenceMatrix = csr_matrix((occValues, (occRow, occCol)), shape=((len(wordIndex), len(descDict))), dtype=np.float64)
save_sparse_csr("occ_matrix", occurenceMatrix)

In [ ]:
mostFreqWord = occurenceMatrix.max(axis=0).data
TF = occurenceMatrix/mostFreqWord
IDF = -np.log((occurenceMatrix != 0).sum(1)/len(descDict))

print(TF.shape, IDF.shape)
TFIDF = np.multiply(TF, IDF)

print(TFIDF.shape)
np.save("TFIDF", TFIDF)

In [ ]:
# ixIndex = descDict['COM-308'][0]
# ixLine = occurenceMatrix[ixIndex]
# ixWordsFreq = dict() #for each word, (TF, DF, score)
# for ixWord in descDict['COM-308'][2]:
#     wordRow = wordIndex[ixWord]
#     freq = occurenceMatrix[wordRow]
#     ixWordsFreq[ixWord] = (occurenceMatrix[wordRow,ixIndex], freq)
    
# print(ixWordsFreq)

## Exercise 4.2: Term-document matrix

## Exercise 4.3: Document similarity search